In [93]:
import git
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from models import TimeSeries

In [94]:
# read our data
confirmed_df =pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

labels = ['Confirmed','Deaths','Recovered']
dfs = [confirmed_df,deaths_df,recovered_df]
data_dict = dict(zip(labels,dfs))

# get last update date:
last_update = TimeSeries(confirmed_df).get_last_update_date()

## Actual Information per Country ##
actual_confirmed = TimeSeries(confirmed_df).get_last_values()
actual_deaths = TimeSeries(deaths_df).get_last_values()
actual_recovered = TimeSeries(recovered_df).get_last_values()

actual_data = pd.DataFrame({'Confirmed':actual_confirmed.values,
                   'Deaths':actual_deaths.values,
                   'Recovered':actual_recovered.values},
                          index=actual_confirmed.index)

# Welcome to this Coronavirus Global Visualizations and Information notebook.

The data source of this notebook is the [repository](https://github.com/CSSEGISandData/COVID-19) of the [Johns Hopkins University](https://www.jhu.edu/). I would like to thank them for making this data available and public.


In this notebook we will be answering the following questions:

* **How is the current situation COVID-19 in the world? (numbers and visualizations)**


* **How does the number of Confirmed/Deaths/Recovered cases behave through time? How does the Mortality Rate evolves through time?**


* **Which are the countries with more confirmed/deaths/recovered cases and also which are the ones with the highest and lowest Mortality Rates?**


* **How the daily new confirmed/deaths/recovered cases are behaving through time? Are we managing the situation? Does the daily increases are decresing or increasing over time? In which date there have been more daily confirmed/deaths/recovered cases?**


* **How can we compare country by country growth?**


* **What is the real form of the cumulated confirmed cases curve, exponential/logistic?**


* **How can we visualize if a country is still in an exponential curve or if it has decreased significantly the new confirmed cases by now?**


> **Note:** In this notebook I'll be using a self-made python module called **models**, for cleaning and displaying the Time Series of the data source for every country and make it easy to work with.

# Current global and country by country information

In [221]:
def get_doubling_times(countries,kind='Confirmed'):
    data = TimeSeries(data_dict[kind]).data

    doubling_times = []
    for country in countries:
        last_week_cases = data.loc[country].values[-8]
        current_cases = data.loc[country].values[-1]

        if last_week_cases>0:
            ratio = current_cases/last_week_cases
            if ratio != 1:
                doubling_time = 7*np.log(2)/np.log(ratio)
                doubling_times.append(doubling_time)
            else:
                doubling_times.append(0)
        else:
            doubling_times.append(0)
        
    return doubling_times

actual_data['Mortality Rates']= actual_data['Deaths']/actual_data['Confirmed']
actual_data.to_csv('exports/daily-data-per-country/'+last_update+'.csv')

print('Global: ',last_update)
print('\tConfirmed: {:,}'.format(actual_data['Confirmed'].sum()))
print('\tDeaths: {:,}'.format(actual_data['Deaths'].sum()))
print('\tRecovered: {:,}'.format(actual_data['Recovered'].sum()))
print('\tMortality Rate: {:.2f}%'.format(100*actual_data['Deaths'].sum()/actual_data['Confirmed'].sum()))
print('--------------------------------')

actual_data.sort_values(by='Confirmed',ascending=False,inplace=True)

for i in range(len(actual_data)):
    print(str(i+1)+'.- '+actual_data.index.to_list()[i]+':')
    print('\tConfirmed: {:,}'.format(actual_data['Confirmed'][i]))
    dbt = get_doubling_times([actual_data.index.to_list()[i]],kind='Confirmed')[0]
    print('\tConfirmed Cases Doubling Time: {:.2f}'.format(dbt),' (Considering the last week increment)')
    print('\tDeaths: {:,}'.format(actual_data['Deaths'][i]))
    print('\tRecovered: {:,}'.format(actual_data['Recovered'][i]))
    print('\tMortality Rate: {:.2f}%'.format(actual_data['Mortality Rates'][i]*100))
    print('--------------------------------')

Global:  2020-04-07
	Confirmed: 1,426,096
	Deaths: 81,865
	Recovered: 300,054
	Mortality Rate: 5.74%
--------------------------------
1.- US:
	Confirmed: 396,223
	Confirmed Cases Doubling Time: 6.52  (Considering the last week increment)
	Deaths: 12,722
	Recovered: 21,763
	Mortality Rate: 3.21%
--------------------------------
2.- Spain:
	Confirmed: 141,942
	Confirmed Cases Doubling Time: 12.38  (Considering the last week increment)
	Deaths: 14,045
	Recovered: 43,208
	Mortality Rate: 9.89%
--------------------------------
3.- Italy:
	Confirmed: 135,586
	Confirmed Cases Doubling Time: 19.55  (Considering the last week increment)
	Deaths: 17,127
	Recovered: 24,392
	Mortality Rate: 12.63%
--------------------------------
4.- France:
	Confirmed: 110,065
	Confirmed Cases Doubling Time: 6.61  (Considering the last week increment)
	Deaths: 10,343
	Recovered: 19,523
	Mortality Rate: 9.40%
--------------------------------
5.- Germany:
	Confirmed: 107,663
	Confirmed Cases Doubling Time: 11.98  (

	Confirmed Cases Doubling Time: 5.01  (Considering the last week increment)
	Deaths: 6
	Recovered: 150
	Mortality Rate: 0.29%
--------------------------------
46.- Dominican Republic:
	Confirmed: 1,956
	Confirmed Cases Doubling Time: 8.55  (Considering the last week increment)
	Deaths: 98
	Recovered: 36
	Mortality Rate: 5.01%
--------------------------------
47.- Greece:
	Confirmed: 1,832
	Confirmed Cases Doubling Time: 14.60  (Considering the last week increment)
	Deaths: 81
	Recovered: 269
	Mortality Rate: 4.42%
--------------------------------
48.- Colombia:
	Confirmed: 1,780
	Confirmed Cases Doubling Time: 7.18  (Considering the last week increment)
	Deaths: 50
	Recovered: 100
	Mortality Rate: 2.81%
--------------------------------
49.- South Africa:
	Confirmed: 1,749
	Confirmed Cases Doubling Time: 18.90  (Considering the last week increment)
	Deaths: 13
	Recovered: 95
	Mortality Rate: 0.74%
--------------------------------
50.- Argentina:
	Confirmed: 1,628
	Confirmed Cases Doubli

	Confirmed Cases Doubling Time: 7.27  (Considering the last week increment)
	Deaths: 3
	Recovered: 41
	Mortality Rate: 0.86%
--------------------------------
93.- Honduras:
	Confirmed: 305
	Confirmed Cases Doubling Time: 6.29  (Considering the last week increment)
	Deaths: 22
	Recovered: 6
	Mortality Rate: 7.21%
--------------------------------
94.- Malta:
	Confirmed: 293
	Confirmed Cases Doubling Time: 8.82  (Considering the last week increment)
	Deaths: 0
	Recovered: 5
	Mortality Rate: 0.00%
--------------------------------
95.- Ghana:
	Confirmed: 287
	Confirmed Cases Doubling Time: 8.39  (Considering the last week increment)
	Deaths: 5
	Recovered: 31
	Mortality Rate: 1.74%
--------------------------------
96.- San Marino:
	Confirmed: 279
	Confirmed Cases Doubling Time: 28.99  (Considering the last week increment)
	Deaths: 34
	Recovered: 40
	Mortality Rate: 12.19%
--------------------------------
97.- Niger:
	Confirmed: 278
	Confirmed Cases Doubling Time: 2.08  (Considering the last 

	Confirmed Cases Doubling Time: 4.57  (Considering the last week increment)
	Deaths: 1
	Recovered: 5
	Mortality Rate: 3.85%
--------------------------------
139.- Haiti:
	Confirmed: 25
	Confirmed Cases Doubling Time: 9.50  (Considering the last week increment)
	Deaths: 1
	Recovered: 0
	Mortality Rate: 4.00%
--------------------------------
140.- Tanzania:
	Confirmed: 24
	Confirmed Cases Doubling Time: 20.77  (Considering the last week increment)
	Deaths: 1
	Recovered: 5
	Mortality Rate: 4.17%
--------------------------------
141.- Burma:
	Confirmed: 22
	Confirmed Cases Doubling Time: 12.67  (Considering the last week increment)
	Deaths: 1
	Recovered: 0
	Mortality Rate: 4.55%
--------------------------------
142.- Libya:
	Confirmed: 20
	Confirmed Cases Doubling Time: 7.00  (Considering the last week increment)
	Deaths: 1
	Recovered: 1
	Mortality Rate: 5.00%
--------------------------------
143.- Antigua and Barbuda:
	Confirmed: 19
	Confirmed Cases Doubling Time: 4.86  (Considering the l

	Confirmed Cases Doubling Time: 0.00  (Considering the last week increment)
	Deaths: 0
	Recovered: 0
	Mortality Rate: 0.00%
--------------------------------


In [102]:
def plot_global_data():
    
    confirmed = TimeSeries(confirmed_df).data.groupby(lambda x: True).sum().transpose()
    deaths = TimeSeries(deaths_df).data.groupby(lambda x: True).sum().transpose()
    recovered = TimeSeries(recovered_df).data.groupby(lambda x: True).sum().transpose()
    actual_infected = confirmed - recovered - deaths
    df = pd.concat([confirmed,deaths,recovered,actual_infected],axis=1)
    df.columns = ['Confirmed','Deaths','Recovered','Actual Infected']
    
    fig = go.Figure()
    annotations=[]
    for col in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col, mode='lines+markers'))
        annotations.append(dict(x=df.index.to_list()[-1],y=df[col][-1],xref="x",
                                yref="y",text='{:,}'.format(int(df[col][-1])),
                                showarrow=True,arrowhead=7))
    
    fig.update_layout(title="<b>Global Cases</b>",xaxis_title="Date",yaxis_title=None,
                      font=dict(family="Arial, bold",size=14),annotations=annotations,
                      legend=dict(x=.01, y=.99))

    fig.show()
    
plot_global_data()

# Global Mortality Rate

This plot shows the evolution of the **Global Mortality Rate** since January 22th, 2020.

In [103]:
def plot_line_mortality_rate():
    
    confirmed = TimeSeries(confirmed_df).data.groupby(lambda x: True).sum().transpose()
    deaths = TimeSeries(deaths_df).data.groupby(lambda x: True).sum().transpose()
    mort_rates = deaths/confirmed
    mort_rates.columns = ['Mortality Rate']
    n = len(mort_rates)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mort_rates.index,y=mort_rates['Mortality Rate'],
                            mode='lines+markers',name='Mortality Rate'))
    fig.add_trace(go.Scatter(x=mort_rates.index,
                             y=[mort_rates['Mortality Rate'].mean()]*len(mort_rates),
                            mode='lines',name='Mean'))
    annotations = []
    for i in range(len(mort_rates['Mortality Rate'])):
        if (i)%3==0:
            annotations.append(dict(x=mort_rates.index.to_list()[n-i-1],
                                    y=mort_rates['Mortality Rate'][n-i-1],
                                    xref="x",yref="y",
                                    text='{:.1f}%'.format(mort_rates['Mortality Rate'][n-1-i]*100),
                                    yshift=0))

    annotations.append(dict(x=mort_rates.index.to_list()[-1],
                            y=mort_rates['Mortality Rate'].mean(),
                            xref="x",yref="y",
                            text='{:.2f}%'.format(mort_rates['Mortality Rate'].mean()*100),
                            yshift=0))
    
    fig.update_layout(title="<b>Global Mortality Rate</b>",
                      xaxis_title="Date",yaxis_title=None,
                      annotations=annotations,
                      legend=dict(x=.01, y=.99))

    fig.show()

plot_line_mortality_rate()

# Global COVID-19 Top Countries

Here are the bar plots, showing the countries with the highest number of the following:

* Confirmed cases
* Deaths cases
* Mortality rates
* Recovered cases

In [217]:
def plot_bar_top(n, kind, ascending=False):
    
    def plot_mortality_rate():
        confirmed = TimeSeries(data_dict['Confirmed']).get_last_values()
        deaths = TimeSeries(data_dict['Deaths']).get_last_values()
        df = pd.DataFrame({'Confirmed':confirmed,'Deaths':deaths,'Mortality Rate':deaths/confirmed})
        df = df[df['Confirmed']>250]
        mortality_rates = df['Mortality Rate']
        top_values = mortality_rates.sort_values(ascending=ascending).head(n)
        top_values = top_values[::-1]
        
        title = "Top {} {}'s  by {}".format(n,kind,last_update)
        xaxis_title = '{}'.format(kind)
        
        fig = px.bar(x=top_values.values,y=top_values.index,color=top_values.index,
                     orientation='h',text=top_values.values)
        fig.update_layout(showlegend=False,title=title,xaxis_title=xaxis_title,
                         yaxis_title='Country')
        fig.update_traces(textposition='inside',texttemplate='%{x:.3p}')
        fig.show()
    
    if kind == 'Mortality Rate':
        plot_mortality_rate()
        
    else:
        data_source = data_dict[kind]
        data = TimeSeries(data_source).get_last_values()

        top_values = list(data.sort_values(ascending=ascending).head(n).values)
        others_sum = data.sort_values().head(len(data)-n).sum()
        top_values.append(others_sum)

        y = data.sort_values(ascending=ascending).head(n).index.to_list()
        y.append('Others')
        y=y[::-1]
        x = top_values[::-1]

        title = 'Top {} {} cases by {}'.format(n,kind,last_update)
        xaxis_title = '{} Cases'.format(kind)

        fig = px.bar(x=x,y=y,color=y,orientation='h',text=x)
        fig.update_layout(showlegend=False,title=title,xaxis_title=xaxis_title,
                         yaxis_title='Country')
        fig.update_traces(textposition='inside',texttemplate='%{x:,}')
        fig.show()

plot_bar_top(15,'Confirmed')
plot_bar_top(15,'Deaths')
plot_bar_top(15,'Mortality Rate')
plot_bar_top(15,'Recovered')

# Countries with the lowest mortality rates

> **Note:** Taking in consideration only the countries that have 250 or more confirmed cases

In [108]:
# Countries with the lowest mortality rates
plot_bar_top(15,'Mortality Rate',ascending=True)

# Global Daily New Cases

This are the global new cases through the time since January 22th, 2020. This are divided in 3:

* Confirmed
* Deaths
* Recovered

In [216]:
def get_diff(array):
    out_arr = []
    for i in range(len(array)):
        if i==0:
            out_arr.append(array[i])
        
        else:
            out_arr.append(array[i]-array[i-1])
    
    return np.array(out_arr)

def plot_daily_diff(kind='Confirmed', country=None):
    
    # if we want to plot global daily new cases or
    # daily new cases of a specific country
    if country==None:
        cases = TimeSeries(data_dict[kind]).data.groupby(lambda x: 'Global').sum().loc['Global'].values
        dates = TimeSeries(data_dict[kind]).data.columns.to_list()
        last_date = max(dates)
        title = 'Global '+kind + " Daily New Cases" + ' by '+ str(last_date)
    else:
        cases = TimeSeries(data_dict[kind],country).data.loc[country].values
        dates = TimeSeries(data_dict[kind],country).data.columns.to_list()
        last_date = max(dates)
        title = kind + ' Daily New Cases in ' + country + ' by '+ str(last_date)
    
    daily_cases = get_diff(cases)
    
    
    fig = px.bar(x=dates,y=daily_cases,color=daily_cases)
    
    annotations = [dict(x=dates[-1],y=daily_cases[-1],xref='x',yref='y',
                       text='{:,}'.format(int(daily_cases[-1])))]
    
    max_value = daily_cases.max()
    max_value_ix = np.where(daily_cases == daily_cases.max())[0][0]
    
    if max_value > daily_cases[-1]:
        annotations.append(dict(x=dates[max_value_ix],y=max_value,xref='x',yref='y',
                                text='Max: {:,}<br>{}'.format(int(max_value),dates[max_value_ix])))
    
    xaxis_title = 'Dates'
    yaxis_title = kind + ' new cases'
    fig.update_layout(title=title,annotations=annotations,xaxis_title=xaxis_title,
                     yaxis_title=yaxis_title)
    
    fig.show()

    
plot_daily_diff(kind='Confirmed')
plot_daily_diff(kind='Deaths')
plot_daily_diff(kind='Recovered')
# plot_daily_diff(kind='Deaths',country='Mexico')
# plot_daily_diff(kind='Deaths',country='US')

# Country by country cumulated confirmed cases trajectories

In [174]:
def plot_countries_trajectories(countries, kind='Confirmed'):
    
    data = TimeSeries(data_dict[kind]).get_data_frame(transpose=True)
    data = data[data>100]
    
    cases = []
    for country in countries:
        values = list(data[country].dropna().values)
        if len(values)>50:
            cases.append(values[:49])
        else:
            cases.append(values)
    
    cases_dict = dict(zip(countries,cases))
    
    fig = go.Figure()
    annot = []
    for country in countries:
        fig.add_trace(go.Scatter(y=cases_dict[country],name=country,mode='lines+markers',marker_size=4, opacity=0.8))
        annot.append(dict(x=cases_dict[country].index(cases_dict[country][-1]),
                          y=np.log10(cases_dict[country][-1]),text=country,
                          showarrow=False,yshift=8))
    
    every_1_days = [(2**x)*100 for x in range(1,9)]
    every_2_days = [(2**(x/2))*100 for x in range(1,23)]
    every_3_days = [(2**(x/3))*100 for x in range(1,40)]
    every_8_days = [(2**(x/8))*100 for x in range(1,50)]
    dbt = [every_1_days,every_2_days,every_3_days,every_8_days]
    dbt_names = ['Every Day','Every 2 Days','Every 3 Days', 'Every week']
    dbt_dict = dict(zip(dbt_names,dbt))
    
    fig.add_trace(go.Scatter(y=every_1_days,name='Every Day',
                             opacity=0.5,mode='lines',marker=dict(color='black')))
    fig.add_trace(go.Scatter(y=every_2_days,name='Every 2 Days',
                             opacity=0.5,mode='lines',marker=dict(color='black')))
    fig.add_trace(go.Scatter(y=every_3_days,name='Every 3 Days',
                             opacity=0.5,mode='lines',marker=dict(color='black')))
    fig.add_trace(go.Scatter(y=every_8_days,name='Every week',
                             opacity=0.5,mode='lines',marker=dict(color='black')))
    
    for label in dbt_names:
        text = 'Cases Double<br>'+label
        annot.append(dict(x=dbt_dict[label].index(dbt_dict[label][-1]),y=np.log10(dbt_dict[label][-1]),
                               text=text,showarrow=False,yshift=15))
    title='<b>Country by Country: How COVID-19 '+kind+' cases trajectories compare</b>'
    fig.update_layout(title=title,yaxis_type='log',xaxis_type='linear',annotations=annot,showlegend=False,
                     xaxis_title='Number of days since 100th case',yaxis_title='Number of cumulated confirmed cases')
    fig.show()

countries = actual_data.head(10).index.to_list()
countries.append('Mexico')
plot_countries_trajectories(countries=countries, kind='Confirmed')

# How can we measure the velocity of the growth in every country? and how we can tell if a country is still in an exponential growth curve or not?

### The answer to this question is:
We plot **new confirmed cases** vs **cumulated confirmed cases** on a logaritmic scale.


* **In a pandemic, how do the growth of confirmed cases behaves?**
<br><br>
Nowadays we hear a lot in the news or articles in the web that the confirmed cases of COVID-19 are growing exponentially in several countries and in all of the world. But, we can't have an exponential growth forever, just because of the simple fact that the number of people, that can be infected in all the world, is limited. That is, if in one point in time all of the humans get infected with the virus, there wont be more confirmed cases because there aren't more people to infect.
<br><br>
I'm not saying that we have to get to that point in order to don't have any more confirmed cases, we can also make a vaccine or the countries can implement social distancing policies in order to decrease the growth of the confirmed cases.
<br><br>
Having said that, I can introduce you the real curve that the confirmed cases growth follows: **Logistic Curve**.
This logistic curve at the beginning is like an exponential curve but, at some point, there is an inflection point where the curve passes from exponential to a kind of logaritmic curve, as you can see in the image below.

![Logistic Curve Example](data/img/logistic_example.png)


* **Why logaritmic scale?**
<br><br>
In the first stages of the pandemic in every country, the growth of the cumulative is most likely exponential, also we have that ``log`` is the inverse function of the ``exp`` function, so if we apply ``log`` to the exponential growth we get a 'linear' curve.
<br><br>
This help us to **visualize and compare better the growths between the countries**, regardless of the big difference of number of cases between one country and another.


* **Is my country still going through an exponential growth?**
<br><br>
When we plot the new confirmed cases vs. cumulated confirmed cases, if the growth is exponential, then in the logaritmic scale we will have a linear curve. So, **if a country has a linear curve that means that the growth is still exponential.**<br><br>
In the other hand, **when a country curve is not a straight line** in this plot; then, we have that the country has gone through that inflection point when the curve passes from exponential to logaritmic. So, in this case, we have that the growth of confirmed cases is no longer an exponential growth.



> **Note:** In this plots, we take the weekly increments of confirmed cases, not the daily, in order to be a pessimist plot and not take in consideration small decrements of confirmed cases. This means, that a country line will be no longer exponential only if in the last week, the new confirmed cases had decreased significantly compared with the previous week.

# New Confirmed Cases vs. Cumulated Confirmed Cases
Of the 10 countries with more confirmed cases in the world, and Mexico.

In [176]:
def plot_trajectory(countries,days=8,cases_limit=50,log=True):
    """
    """
    fig = go.Figure()
    
    annotations=[]
    for country_name in countries:
        confirmed = TimeSeries(confirmed_df,country_name).get_data_frame(transpose=True)
        confirmed = confirmed[confirmed[country_name]>cases_limit][country_name].values[::-1]
        confirmed = confirmed[::days]
        confirmed = confirmed[::-1]
        deltas = get_diff(confirmed)

        fig.add_trace(go.Scatter(x=confirmed,y=deltas,name=country_name))
        
        if log:
            annotations.append(dict(x=np.log10(confirmed[-1]),
                                   y=np.log10(deltas[-1]),
                                   text=country_name,showarrow=False,
                                   xshift=10,yshift=10))
            
        
        else:
            annotations.append(dict(x=(confirmed[-1]),
                                   y=(deltas[-1]),
                                   text=country_name,showarrow=False,
                                   xshift=10,yshift=10))
    if log:
        subtitle ='On a logaritmic scale to normalize the exponential growth. Last Update: '+last_update
        xaxis_type="log"
        yaxis_type="log"
    else:
        subtitle ='On a linear scale. Last Update: '+last_update
        xaxis_type="linear"
        yaxis_type="linear"
    
    annotations.append(dict(xref='paper',yref='paper',x=-.04, y=+1.10,showarrow=False,
                            text =subtitle))

    fig.update_layout(xaxis_type=xaxis_type, yaxis_type=yaxis_type,annotations=annotations,showlegend=False,
                     title = '<b>Trayectory of COVID-19 Confirmed Cases</b>',
                     yaxis_title = 'New Confirmed Cases (in the last {} days)'.format(days),
                     xaxis_title = 'Total Confirmed Cases')
    fig.show()

    
top_confirmed_countries = actual_confirmed.sort_values(ascending=False).head(10).index.to_list()
top_confirmed_countries.append('Mexico')
plot_trajectory(top_confirmed_countries,days=8,cases_limit=50,log=True)

# New Confirmed Cases vs. Cumulated Confirmed Cases
Of the countries of **Latin America**

In [177]:
latin_america_countries = ['Brazil','Mexico','Argentina','Colombia','Chile','Ecuador','Peru',
                          'Dominican Republic','Costa Rica','Uruguay']

plot_trayectory(latin_america_countries,days=8,cases_limit=10,log=True)

# Which countries are recently increasing faster in COVID-19 confirmed cases?

### Doubling Times
A **doubling time** is the time that takes to a country to double the confirmed cases given the increment of confirmed cases between two dates.

> **Note:** In this case I'll use the weekly increments to calculate the doubling times.

In [214]:
countries = TimeSeries(confirmed_df).data.index.to_list()
db_times = get_doubling_times(countries=countries,kind='Confirmed')
print('Days that take a country to double the confirmed COVID-19 cases:')
dict(zip(countries,db_times))

Days that take a country to double the confirmed COVID-19 cases:


{'Afghanistan': 5.46204893105106,
 'Albania': 10.664422818167287,
 'Algeria': 6.757927811851233,
 'Andorra': 13.071320140618878,
 'Angola': 5.468288958335917,
 'Antigua and Barbuda': 4.859178941663236,
 'Argentina': 11.160254625769044,
 'Armenia': 10.277198117200411,
 'Australia': 18.879404631058208,
 'Austria': 22.42549255933666,
 'Azerbaijan': 5.526341436469175,
 'Bahamas': 5.658672752871219,
 'Bahrain': 13.556612568606521,
 'Bangladesh': 4.153990412213938,
 'Barbados': 7.866785364811225,
 'Belarus': 2.7978267444824354,
 'Belgium': 8.78462884072002,
 'Belize': 5.726475293708761,
 'Benin': 4.5736247574923965,
 'Bhutan': 21.743986036537727,
 'Bolivia': 8.15427082887004,
 'Bosnia and Herzegovina': 8.109517313306847,
 'Botswana': 11.966579039460182,
 'Brazil': 5.40291851027201,
 'Brunei': 106.7262843715021,
 'Bulgaria': 13.15338074324298,
 'Burkina Faso': 12.566050208255199,
 'Burma': 12.668742605986399,
 'Burundi': 11.966579039460182,
 'Cabo Verde': 31.47589273921344,
 'Cambodia': 90.54

## Countries that double confirmed cases faster (in the last week).
This countries are increasing their confirmed cases at a faster rate.

This may be explained by the lack of a good health system or not implementing enough social distancing policies.

> **Note:** I take only in consideration those countries with more than 200 confirmed cases up to date.

In [215]:
countries = actual_data[actual_data['Confirmed']>200].index.to_list()
db_times = get_doubling_times(countries=countries,kind='Confirmed')
df = pd.DataFrame(data={'Doubling Time':db_times},index=countries)
df = df[df>0].dropna()
df.sort_values(by='Doubling Time',ascending=True).head(15)

,Doubling Time
Niger,2.080823
Belarus,2.797827
India,3.633247
United Arab Emirates,3.827395
Cameroon,3.955949
Russia,4.162564
Uzbekistan,4.385682
Moldova,4.427942
Peru,4.756028
Serbia,4.850947


## Countries that double confirmed cases slower (in the last week).
This countries are the ones that actually have achieved to decrease the growth rate of confirmed cases.

This may be explained by the fact that theu have been implementing good policies in order to manage the pandemic situation. Or also, this countries have had the virus for a long period of time and is managing to improve the situation.

> **Note:** I take only in consideration those countries with more than 200 confirmed cases up to date.

In [206]:
countries = actual_data[actual_data['Confirmed']>200].index.to_list()
db_times = get_doubling_times(countries=countries,kind='Confirmed')
df = pd.DataFrame(data={'Doubling Time':db_times},index=countries)
df = df[df>0].dropna()
df.sort_values(by='Doubling Time',ascending=False).head(15)

,Doubling Time
China,911.809281
"Korea, South",89.526980
Lebanon,31.600549
Taiwan*,31.295830
Vietnam,30.161821
San Marino,28.988116
Austria,22.425493
Uruguay,21.404043
Italy,19.554291
Jordan,19.152251


# Which countries are recently increasing faster in COVID-19 deaths cases?

## Countries that double deaths cases in less time than others. (In the last week)
This are countries that actually are having deaths cases at a faster rate.

This may be explained because of the health system of the countries or if a country is managing to control the disease propagation. Also, if a country is starting to have more people infected of older age or with an underlying chronic illness, it can increase significantly it's growth rate of deaths.

> **Note:** Taking in consideration only countries that actually have registrated more than 50 deaths.

In [212]:
countries = actual_data[actual_data['Deaths']>50].index.to_list()
db_times = get_doubling_times(countries=countries,kind='Deaths')
df = pd.DataFrame(data={'Doubling Time':db_times},index=countries)
df = df[df>0].dropna()
df.sort_values(by='Doubling Time',ascending=True).head(15)

,Doubling Time
Mexico,3.243099
Algeria,3.281724
India,3.334071
Poland,3.559021
Serbia,3.625558
Canada,3.698742
Peru,3.815595
United Kingdom,3.925684
Brazil,3.952540
Russia,3.953645


## Countries that double deaths cases in more time than others. (In the last week)
This are countries that actually are having deaths cases at a slower rate.

This may be explained because of the health system of the countries or if a country is managing to control the disease propagation. Also if a country is starting to have more people infected of older age or with an underlying chronic illness, it can increase significantly it's growth rate of deaths.

> **Note:** Taking in consideration only countries that actually have registrated more than 50 deaths.

In [209]:
countries = actual_data[actual_data['Deaths']>50].index.to_list()
db_times = get_doubling_times(countries=countries,kind='Deaths')
df = pd.DataFrame(data={'Doubling Time':db_times},index=countries)
df = df[df>0].dropna()
df.sort_values(by='Doubling Time',ascending=False).head(15)

,Doubling Time
China,619.937010
"Korea, South",28.558315
Iraq,18.493488
Iran,16.745556
Italy,15.129302
Malaysia,12.703824
Indonesia,9.993722
Japan,9.773710
Greece,9.653306
Spain,9.580575


# Which countries are recently increasing faster in COVID-19 recovered cases?

## Countries that double recovered cases in less time than others. (In the last week)
This are the countries that actually are recovering people at a faster rate.

> **Note:** Taking in consideration only countries that actually have registrated more than 50 recovered cases.

In [210]:
countries = actual_data[actual_data['Recovered']>50].index.to_list()
db_times = get_doubling_times(countries=countries,kind='Recovered')
df = pd.DataFrame(data={'Doubling Time':db_times},index=countries)
df = df[df>0].dropna()
df.sort_values(by='Doubling Time',ascending=True).head(15)

,Doubling Time
Finland,1.426565
Poland,1.467483
Denmark,1.592399
Mexico,1.675933
Sweden,1.902443
Slovenia,2.089242
Turkey,2.589982
Luxembourg,2.647648
Chile,2.772091
Pakistan,2.803469


## Countries that double recovered cases in more time than others. (In the last week)
This are the countries that actually are recovering people slower rate.


If a country is in this list, not necesarly is because the people there doesn't recover, it can be that they have recovered almost all the people so, that's why they recover less people now.

> **Note:** Taking in consideration only countries that actually have registrated more than 50 recovered cases.

In [211]:
countries = actual_data[actual_data['Recovered']>50].index.to_list()
db_times = get_doubling_times(countries=countries,kind='Recovered')
df = pd.DataFrame(data={'Doubling Time':db_times},index=countries)
df = df[df>0].dropna()
df.sort_values(by='Doubling Time',ascending=False).head(15)

,Doubling Time
China,309.524177
Diamond Princess,185.276317
Netherlands,67.005352
Belarus,34.947814
"Korea, South",22.744006
Japan,14.536909
Argentina,14.170361
Kuwait,13.348055
Taiwan*,12.785673
Italy,11.058782
